In [6]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

import sympy as sp
from sympy import *
import functools 
from IPython.display import display, Math, Latex

#init_printing(use_unicode = True)

e = sp.symbols('e0:1000', commutative=False)

basis = list(e)

def translate(x):#turn usual basis into index basis, to be finished later
    
    return 0

def getweight(x):
    return float((basis.index(x))/2 - 1)

def getindex(x):
    return int(2*(x + 1))

def getparity(x):
    L = unpack(x)
    parity=0
    for term in L:
        parity += int(basis.index(term))
    return int(parity%2)

def unpack(x):#check for multiplicity and write powers as distinct elements in list
    if type(x).__name__ == 'Mul': #check if passed argument is product e.g. e[1]*e[2]
        L = list(x.args)
        for term in L:
            if type(term).__name__ == 'Pow':#check if passed argument is product that contains powers e.g. e[1]*e[2]**2
                J = list(term.args) #returns [vector, power]
                power = J[1]
                base = J[0]
                firstL = L[:L.index(term)] #splitting the list to insert multiplicity many elements
                secondL = L[L.index(term) + 1: ]
                for i in range(power):
                    firstL.append(base)
                L = firstL + secondL
        return L
            
    if type(x).__name__ == 'Pow':#check if passed argument is just a power itself e.g. e[1]**2
        L = list(x.args) #returns [vector, power]
        base = L[0]
        power = L[1]
        L=[]
        for i in range(power):
            L += unpack(base)
        return L
            
    if type(x).__name__ == 'Symbol':#check if passed argument is single bases vector e.g. e[1]
        return [x]
    

def bracket(x,y):
    #bilinearity 
    if type(x).__name__ == 'Add':#split across sums on x
        L = list(x.args)
        J = []
        for term in L:
            J.append( bracket(term, y) )
        return sum(J)
        
    if type(y).__name__ == 'Add':#split across sums on y
        L = list(y.args)
        J = []
        for term in L:
            J.append( bracket(x, term) )
        return sum(J)
    
    if type(x).__name__ == 'Mul':    #pull out x coefficients
        L = list(x.args)
        if L[0].is_number:
            coeff = L[0]
            del L[0]
            return coeff*bracket( prod(L) , y )

    if type(y).__name__ == 'Mul':    #pull out y coefficients
        L = list(y.args)
        if L[0].is_number:
            coeff = L[0]
            del L[0]
            return coeff*bracket( x , prod(L) )
    
    
    #ad homomorphism property
    unpackedx = unpack(x)
    if len(unpackedx) > 1:#reducing product
        xreductionprod = prod(unpackedx[:-1])
        return bracket(xreductionprod, bracket(unpackedx[-1], y))
    
    #super Leibniz
    unpackedy = unpack(y)
    if len(unpackedy) > 1:
        yreductionprod = prod(unpackedy[1:])
        leibniz1 = bracket(x, unpackedy[0]) * yreductionprod
        leibniz2 = unpackedy[0] * bracket(x, yreductionprod)
        derivationParity = int((-1)**(getparity(x) * getparity(unpackedy[0])))
        return leibniz1 + derivationParity*leibniz2
        
    
    #base case
    weightx = getweight(x)
    weighty = getweight(y)
    combinedweight = weightx + weighty
    combinedindex = getindex(combinedweight)
    
    if weightx.is_integer() and weighty.is_integer():
        z = (weighty - weightx)*e[combinedindex]
    if weightx.is_integer() and not weighty.is_integer():
        z = (weighty - weightx/2)*e[combinedindex]
    if not weightx.is_integer() and weighty.is_integer():
        z = -1*(weightx - weighty/2)*e[combinedindex]
    if not weightx.is_integer() and not weighty.is_integer():
        z = 2*e[combinedindex]
    return z

def standardize(x):
    if type(x).__name__ == 'Symbol' or x.is_number:
        return x
    
    if type(x).__name__ == 'Add':#split across sums on x
        L = list(x.args)
        J = []
        for term in L:
            J.append( standardize(term) )
        return sum(J)

    if type(x).__name__ == 'Mul':    #pull out x coefficients
        L = list(x.args)
        if L[0].is_number:
            coeff = L[0]
            del L[0]
            return coeff*standardize( prod(L) )
    
    L = unpack(x)
    ordercheck = 0
    indexforswap = 0
    
    for i in range(len(L)-1):#store index of point for swap, exit loop and then call swap with bracket   
        if basis.index(L[i]) > basis.index(L[i+1]):
            indexforswap = i
            ordercheck+=1
            break
        
        if basis.index(L[i]) == basis.index(L[i+1]) and basis.index(L[i])%2 == 1:
            L[i] = .5*bracket(L[i], L[i+1])
            del L[i+1]
            return standardize( prod(L) )
            
    if ordercheck == 0: 
        return x
    
    if ordercheck != 0:
        firstJ= []
        secondJ = []
        for j in L[:i]:
            firstJ.append(j)
        firstJ.append(bracket(L[i], L[i+1])) #insert bracket into new list
            
        if indexforswap + 2 < len(L):
            for j in L[indexforswap+2:]:
                secondJ.append(j) #get second half of list
                    
        J = firstJ + secondJ
        parity = int( (-1)**( getparity(L[indexforswap] ) * getparity( L[indexforswap+1]) ) )
        L[indexforswap], L[indexforswap+1] = L[indexforswap+1], L[indexforswap] #finally swap

        return parity*standardize( prod(L) ) + standardize( prod(J) ) #run through again to reorder



#######################################################################################################################################################################
#Defining some commonly used objects. Sn is the step operator of weight n. Q and Qs are the Casimir operators of sl2 and osp(1|2), respectively.
S2 = (4*e[2] - 2)*e[2] - 3*e[4]**2 - 3*e[3]*e[5]
Q = e[2]**2 - e[2] - e[4]*e[0]
Lambda = e[2] - e[3]*e[1]
T = Lambda - 1/4
Qs = 2*e[2]**2 + e[2] + e[1]*e[3] - 2*e[0]*e[4]
Z = e[4]*e[1] - 2*e[2]*e[3] + e[0]*e[5]
#######################################################################################################################################################################

In [7]:
b = bracket(e[1], Z)
sb = simplify(standardize(b))
standqs = 2*standardize(Qs)
sbdiff = simplify(sb + standqs)